# pdf 기반의 chatbot 시스템 feat.Finetuned Model

python version : 3.10.12
pytorch : 2.1.0

In [1]:
import os
import torch
import getpass

In [2]:
import chromadb
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from openai import OpenAI

## 환경 구성

In [4]:
# GPU or CPU Device Setting
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [5]:
# OpenAi api-key Setting
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

### 모델 임베딩

사용 가능한 모델 : 추가예정..

In [6]:
embedding = SentenceTransformerEmbeddings(model_name='BM-K/KoSimCSE-roberta-multitask', model_kwargs={'device': device}, encode_kwargs={'normalize_embeddings':True})

No sentence-transformers model found with name /Users/siina/.cache/torch/sentence_transformers/BM-K_KoSimCSE-roberta-multitask. Creating a new one with MEAN pooling.


### Load Vector DB

In [7]:
chroma_client = chromadb.PersistentClient(path="chroma")

collection_name = "vector_db"
collection = chroma_client.get_collection(collection_name)

vectorstore = Chroma(
    client= chroma_client,
    collection_name= collection_name,
    embedding_function= embedding,
    persist_directory="./chroma"
)

In [8]:
print("There are", vectorstore._collection.count(), "in the collection.")

There are 914 in the collection.


In [9]:
retriever = vectorstore.as_retriever()

In [10]:
results = retriever.get_relevant_documents('청소년 상담 센터')

In [11]:
print('Q : 청소년 상담 센터', "\nA :", results[0].metadata['title'])

Q : 청소년 상담 센터 
A : 청소년동반자 프로그램


## 메인 서비스 구현

In [12]:
## 검색 옵션 설정

# 거리 기반 : 뭐라도 일단 매칭은 해줌.
# retriever = vectorstore.as_retriever(search_kwargs={"k": 5}, search_type="mmr")

# 임계점 기반 : 적절한 threshold 값 선정이 필수임.
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.2})

In [13]:
## llm 모델 설정
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4 / gpt-3.5-turbo / gpt-4-1106-preview

### llm prompt

In [14]:
# 검색된 문장 내에서만 대답하도록 하고 내용을 바꾸지 못하게 프롬프트 작성

system_template="""Use the following pieces of context to answer the users question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("source_documents"), use "source_documents" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{context}

You MUST answer in Korean"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": prompt}

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [78]:
query = "연매출 1억 5천만 원 이상의 자영업자인데 국민내일배움카드제 서비스를 받을 수 있나요?"
result = chain(query)
print(result)

{'query': '연매출 1억 5천만 원 이상의 자영업자인데 국민내일배움카드제 서비스를 받을 수 있나요?', 'result': '아니요, 연매출 1억 5천만 원 이상의 자영업자는 국민내일배움카드제 서비스를 받을 수 없습니다.', 'source_documents': [Document(page_content='국민내일배움카드제\n\n대상\n 국민 누구나 신청 가능 (단, 공무원, 사립학교 교직원, 연매출 1억 5천만 원 이상의 자영업자, 월 임금(소득) 300만 원 이상의 대규모기업 근로자(만 45세 미만) 특수형태근로종사자, 졸업까지 남은 수업 연한이 2년을 초과하는 대학 재학생, 만 75세 이상자는 지원 제외)\n\n내용\n 실업·재직 등 경제활동 상태에 관계없이 계좌발급일로부터 5년간 300~500만 원 내에서 훈련비 지원', metadata={'doc_id': 'a7e7901f-ae02-4988-b12b-fed04189c8c0', 'tag': '청년,고용,취업훈련,실직으로 곤란을 겪고 있을 때(취업지원서비스),중장년', 'title': '국민내일배움카드제'}), Document(page_content='BC카드는 온라인 지마켓, 옥션, 먼슬리씽(앱), 페이북쇼핑, 우리WON마켓, 국민행복몰 지원, 오프라인 홈플러스(익스프레스 포함), 이마트, 이마트트레이더스, 농협하나로마트, CU편의점, GS25편의점, 노브랜드, PK마켓, GS더프레시, 부츠(boots) 지원\n 삼성카드는 온라인 삼성카드쇼핑몰, 국민행복몰 지원, 오프라인 홈플러스(익스프레스 포함), 이마트, 이마트트레이더스, 농협하나로마트, CU편의점, GS25편의점, 노브랜드, PK마켓, GS더프레시, 부츠(boots) 지원\n 롯데카드는 온라인 올마이쇼핑몰 지원, 오프라인 홈플러스(익스프레스 포함), GS25편의점, CU편의점, 농협하나로마트, 롯데마트, VIC마켓 지원\n 신한카드는 온라인 신한카드 올댓쇼핑, 국민행복몰 지원, 오프라인 GS25편의점, CU편의점 지원\n 국민카드는 온라인 KB국

In [79]:
result['source_documents']

[Document(page_content='국민내일배움카드제\n\n대상\n 국민 누구나 신청 가능 (단, 공무원, 사립학교 교직원, 연매출 1억 5천만 원 이상의 자영업자, 월 임금(소득) 300만 원 이상의 대규모기업 근로자(만 45세 미만) 특수형태근로종사자, 졸업까지 남은 수업 연한이 2년을 초과하는 대학 재학생, 만 75세 이상자는 지원 제외)\n\n내용\n 실업·재직 등 경제활동 상태에 관계없이 계좌발급일로부터 5년간 300~500만 원 내에서 훈련비 지원', metadata={'doc_id': 'a7e7901f-ae02-4988-b12b-fed04189c8c0', 'tag': '청년,고용,취업훈련,실직으로 곤란을 겪고 있을 때(취업지원서비스),중장년', 'title': '국민내일배움카드제'}),
 Document(page_content='BC카드는 온라인 지마켓, 옥션, 먼슬리씽(앱), 페이북쇼핑, 우리WON마켓, 국민행복몰 지원, 오프라인 홈플러스(익스프레스 포함), 이마트, 이마트트레이더스, 농협하나로마트, CU편의점, GS25편의점, 노브랜드, PK마켓, GS더프레시, 부츠(boots) 지원\n 삼성카드는 온라인 삼성카드쇼핑몰, 국민행복몰 지원, 오프라인 홈플러스(익스프레스 포함), 이마트, 이마트트레이더스, 농협하나로마트, CU편의점, GS25편의점, 노브랜드, PK마켓, GS더프레시, 부츠(boots) 지원\n 롯데카드는 온라인 올마이쇼핑몰 지원, 오프라인 홈플러스(익스프레스 포함), GS25편의점, CU편의점, 농협하나로마트, 롯데마트, VIC마켓 지원\n 신한카드는 온라인 신한카드 올댓쇼핑, 국민행복몰 지원, 오프라인 GS25편의점, CU편의점 지원\n 국민카드는 온라인 KB국카몰, 국민행복몰 지원, 오프라인 GS25편의점, CU편의점 지원', metadata={'doc_id': '2f60835c-103e-4101-984b-346ce7a0fcb7', 'tag': '저소득층,교육비가 부담될 때(보건 및 복지 지원),생활지원,아동·청소년',

In [66]:
query = "대학교를 다녀야하는데 돈이 없어요. 우리집은 돈을 지원해줄수 없고 나는 여전히 대학교를 다녀야해요. 어떡하죠?"
result = chain(query)
print(result)

{'query': '대학교를 다녀야하는데 돈이 없어요. 우리집은 돈을 지원해줄수 없고 나는 여전히 대학교를 다녀야해요. 어떡하죠?', 'result': '학자금 융자나 장학금을 신청해보는 것이 좋을 것 같습니다. 농촌출신 대학생을 위한 학자금 융자사업이나 푸른등대 삼성기부장학사업 등이 있을 수 있습니다. 또한, 취업 후 상환 학자금대출도 고려해볼 수 있습니다. 자세한 내용은 한국장학재단(전화 1599-2290, www.kosaf.go.kr)에 문의하거나 모바일 챗봇 상담(https://mo.kosaf.go.kr/apps?location=chatbot)을 이용해보세요. 출처: 농촌출신 대학생 학자금 융자사업, 푸른등대 삼성기부장학사업, 취업 후 상환 학자금대출 (source_documents)', 'source_documents': [Document(page_content='농촌출신 대학생 학자금 융자사업\n\n대상\n 농어촌지역에 주민등록 주소를 두고 6개월(180일) 이상 거주하고 있거나 농어업에 종사하고 있는 학부모(보호자)의 자녀(대학생) 또는 농어업에 종사하는 대학생 본인\n 직전학기 성적 70점/100점(C학점) 이상, 12학점 이상 이수한 대학생\n   소속대학의 최저 이수학점이 12학점 미만인 경우는 대학 학사규정에 의함\n   신입생군(신입생, 편입생, 재입학생), 장애인인 경우 성적, 이수학점 기준 적용 제외\n   졸업 학년 학생의 경우 이수학점 기준 적용 제외\n\n1순위의 세부 자격 요건은 농어업에 종사하는 학부모(보호자)의 자녀(대학생), 농어업에 종사하는 대학생 본인, 기초생활수급자, 차상위계층, 장애인, 다문화가족, 다자녀가구(3자녀 이상)\n 2순위의 세부 자격 요건은 농어촌지역에 거주하는 학부모(보호자)의 자녀(대학생)\n 3순위의 세부 자격 요건은 특별승인자\n\n내용\n 당해 학기 대학이 통보한 등록금(입학금, 수업료 등) 범위 내 전액 무이자 대출', metadata={'doc_id': '151f20a4-7a77-4f5f-9

In [67]:
result['source_documents']

[Document(page_content='농촌출신 대학생 학자금 융자사업\n\n대상\n 농어촌지역에 주민등록 주소를 두고 6개월(180일) 이상 거주하고 있거나 농어업에 종사하고 있는 학부모(보호자)의 자녀(대학생) 또는 농어업에 종사하는 대학생 본인\n 직전학기 성적 70점/100점(C학점) 이상, 12학점 이상 이수한 대학생\n   소속대학의 최저 이수학점이 12학점 미만인 경우는 대학 학사규정에 의함\n   신입생군(신입생, 편입생, 재입학생), 장애인인 경우 성적, 이수학점 기준 적용 제외\n   졸업 학년 학생의 경우 이수학점 기준 적용 제외\n\n1순위의 세부 자격 요건은 농어업에 종사하는 학부모(보호자)의 자녀(대학생), 농어업에 종사하는 대학생 본인, 기초생활수급자, 차상위계층, 장애인, 다문화가족, 다자녀가구(3자녀 이상)\n 2순위의 세부 자격 요건은 농어촌지역에 거주하는 학부모(보호자)의 자녀(대학생)\n 3순위의 세부 자격 요건은 특별승인자\n\n내용\n 당해 학기 대학이 통보한 등록금(입학금, 수업료 등) 범위 내 전액 무이자 대출', metadata={'doc_id': '151f20a4-7a77-4f5f-9c94-c573e80c92a6', 'tag': '교육 대출,서민금융,청년,교육비가 부담될 때(학자금대출 지원),교육,기타위기별·상황별 지원,농어업인·도서민,학자금', 'title': '농촌출신 대학생 학자금 융자사업'}),
 Document(page_content='푸른등대 삼성기부장학사업\n\n대상\n 국내 4년제 및 전문 대학교 재학생 중 장애인 가정, 새터민(탈북주민) 가정, 가정 외 보호시설 출신, 학생가장, 조손가정, 기초생활/차상위계층, 한부모가족, 다문화가정, 주거비지원 필요자, 기타 사회적배려계층 중 신청자격을 충족하여 대학의 추천을 받은 자 (전공대, 대학원대, 방송통신대, 원격대, 사이버대, 기능대, 평생교육원 제외)\n\n내용\n 지원금액은 학기당 150만 원\n 지원기간은 2개 학기(휴학, 편입, 자퇴 등 학적

In [58]:
query = "취업준비를 하는데 처음이라 잘 모르겠어요. 도움이 될만한 제도가 있나요?"
result = chain(query)
print(result)

{'query': '취업준비를 하는데 처음이라 잘 모르겠어요. 도움이 될만한 제도가 있나요?', 'result': '네, 취업준비를 도와주는 다양한 제도가 있습니다. 예를 들어, 취업지원서비스를 통해 취업활동계획에 따라 직업훈련, 일경험, 복지프로그램 등을 제공받을 수 있습니다. 또한, 구직활동지원으로 동행면접, 이력서·면접 컨설팅, 일자리정보 제공, 채용박람회, 취업알선 등의 지원을 받을 수 있습니다. 또한, 허그일자리지원 프로그램을 통해 단계별 맞춤형 취업지원과 수당을 받을 수도 있습니다. 여성새로일하기센터에서도 종합적인 취업지원서비스를 제공하고 있습니다. 이러한 제도들을 활용하여 취업준비에 도움을 받을 수 있습니다. (source_documents)', 'source_documents': [Document(page_content='취업 지원의 유형은 공통, 세부 지원내용은 (취업활동계획) 심층상담을 통해 개인별 역량·의지별 수립, (취업지원서비스) 취업활동계획에  따라 직업훈련 ·일경험 ·복지 프로그램 등 제공, * (취업지원)직업훈련, 일경험, 복지프로그램(생계, 의료, 금융, 돌봄서비스 등) 연계, (구직활동지원) 동행면접, 이력서 ·면접 컨설팅, 일자리정보 제공, 채용박람회, 취업알선 등\n 소득 지원의 유형은 Ⅰ유형, 세부 지원내용은 (구직촉진수당) 취업활동계획에  따른 구직활동  이행 시 월 50~90 만 원, 6개월 지원, * 기본 50만 원 및 부양가족(18세 이하, 70세 이상, 중증장애인) 1인당 10만 원씩 추가, (조기취업성공수당) 취업활동계획  수립 후 3개월 내 조기취업  시 잔여 구직촉진수당의  50%', metadata={'doc_id': '33c7452d-297a-4962-9d60-879250455faa', 'tag': '청년,저소득층,고용,취업훈련,실직으로 곤란을 겪고 있을 때(취업지원서비스),중장년', 'title': '국민취업지원제도'}), Document(page_content='허그일자리지원 프로그램\n\n대상\n 만

In [59]:
result['source_documents']

[Document(page_content='취업 지원의 유형은 공통, 세부 지원내용은 (취업활동계획) 심층상담을 통해 개인별 역량·의지별 수립, (취업지원서비스) 취업활동계획에  따라 직업훈련 ·일경험 ·복지 프로그램 등 제공, * (취업지원)직업훈련, 일경험, 복지프로그램(생계, 의료, 금융, 돌봄서비스 등) 연계, (구직활동지원) 동행면접, 이력서 ·면접 컨설팅, 일자리정보 제공, 채용박람회, 취업알선 등\n 소득 지원의 유형은 Ⅰ유형, 세부 지원내용은 (구직촉진수당) 취업활동계획에  따른 구직활동  이행 시 월 50~90 만 원, 6개월 지원, * 기본 50만 원 및 부양가족(18세 이하, 70세 이상, 중증장애인) 1인당 10만 원씩 추가, (조기취업성공수당) 취업활동계획  수립 후 3개월 내 조기취업  시 잔여 구직촉진수당의  50%', metadata={'doc_id': '33c7452d-297a-4962-9d60-879250455faa', 'tag': '청년,저소득층,고용,취업훈련,실직으로 곤란을 겪고 있을 때(취업지원서비스),중장년', 'title': '국민취업지원제도'}),
 Document(page_content='허그일자리지원 프로그램\n\n대상\n 만 18세 이상~만 64세 이하의 형사처분 또는 보호처분을 받은 사람으로 취업 희망자 (위기 청소년은 만 15세 이상)\n\n내용\n 개인별 취업활동계획에 따라 단계별 맞춤형 취업지원과 함께 각종 수당을 지급하는 통합 취업지원프로그램\n   1단계 : 취업설계\n   심층상담, 직업심리검사, 집단상담(8시간 참여), 취업활동계획수립\n   개인별 취업활동계획을 수립 시 최대 25만 원의 참여수당 지급 (개별상담 15만 원, 집단상담 10만 원) (단, 출소 예정자는 미지급)\n\n2단계 : 직업능력개발\n    개별 취업활동계획에 따른 취·창업 교육 (공단 지원 훈련, 직영훈련(기술교육원), 국민내일배움카드(고용노동부 연계), 취업매칭반 훈련)\n    교육비 최대 300만 원, 훈련참여지원수당

In [55]:
query = "우리집은 가난해. 우리 집에 화재가 나서 화상을 입었어. 위로금 같은걸 받을 수 있을까?"
result = chain(query)
print(result)

{'query': '우리집은 가난해. 우리 집에 화재가 나서 화상을 입었어. 위로금 같은걸 받을 수 있을까?', 'result': '네, 가난한 가정이 화재로 인해 피해를 입은 경우, 긴급복지 지원제도를 통해 위로금을 받을 수 있습니다. 긴급복지 지원제도는 주소득자가 화재 또는 자연재해 등으로 인해 생계유지가 곤란한 경우에 지원하는 제도입니다. 자세한 내용은 보건복지상담센터(전화 129)에 문의하시면 도움을 받을 수 있습니다. (source_documents)', 'source_documents': [Document(page_content='위기 상황의 지원대상은 ① 주소득자의 사망, 가출, 행방불명, 구금시설 수용 등으로 소득을 상실한 경우 ②중한 질병 또는 부상을 당한 경우 ③가구 구성원으로부터 방임 또는 유기되거나 학대 등을 당한 경우 ④ 가족폭력을 당하여 가구구성원과 함께 원만한 가정 생활을 하기 곤란하거나 가구구성원으로부터 성폭력을 당한 경우 ⑤ 화재 또는 자연재해 등으로 인해 거주하는 주택 또는 건물에서 생활하기 곤란한 경우 ⑥ 주소득자 또는 부소득자의 휴업, 폐업 또는 사업장의 화재 등으로 영업이 곤란하게 된 경우 ⑦ 주소득자 또는 부소득자의 실직으로 소득을 상실한 경우 ⑧지자체 조례로 정한 사유가 발생한 경우(소득활동 미미(가구원 간호, 간병, 양육), 기초수급 중지, 미결정, 수도·가스 중단, 사회보험료·주택임차료 체납 등)는 관할 지방자치단체의 조례 확인 ⑨그 밖에 보건복지부장관이 정하여 고시하는 경우 (이혼, 단전, 교정시설 출소, 노숙, 복지사각지대 발굴대상자·통합사례관리 대상자·자살고위험군으로 생계가 어렵다고 추천을', metadata={'doc_id': '29ab5a35-eb95-453b-9c14-6313da58c817', 'tag': '영유아,노령층,건강,생활지원,아동·청소년,청년,한부모,1인가구,가족이 특별한 상황에 처했을 때 (다문화가족 지원),저소득층,주거,다문화·북한이탈주민,임신·출산,장애인,중장년', 'title': '긴급복

In [56]:
result['source_documents']

[Document(page_content='위기 상황의 지원대상은 ① 주소득자의 사망, 가출, 행방불명, 구금시설 수용 등으로 소득을 상실한 경우 ②중한 질병 또는 부상을 당한 경우 ③가구 구성원으로부터 방임 또는 유기되거나 학대 등을 당한 경우 ④ 가족폭력을 당하여 가구구성원과 함께 원만한 가정 생활을 하기 곤란하거나 가구구성원으로부터 성폭력을 당한 경우 ⑤ 화재 또는 자연재해 등으로 인해 거주하는 주택 또는 건물에서 생활하기 곤란한 경우 ⑥ 주소득자 또는 부소득자의 휴업, 폐업 또는 사업장의 화재 등으로 영업이 곤란하게 된 경우 ⑦ 주소득자 또는 부소득자의 실직으로 소득을 상실한 경우 ⑧지자체 조례로 정한 사유가 발생한 경우(소득활동 미미(가구원 간호, 간병, 양육), 기초수급 중지, 미결정, 수도·가스 중단, 사회보험료·주택임차료 체납 등)는 관할 지방자치단체의 조례 확인 ⑨그 밖에 보건복지부장관이 정하여 고시하는 경우 (이혼, 단전, 교정시설 출소, 노숙, 복지사각지대 발굴대상자·통합사례관리 대상자·자살고위험군으로 생계가 어렵다고 추천을', metadata={'doc_id': '29ab5a35-eb95-453b-9c14-6313da58c817', 'tag': '영유아,노령층,건강,생활지원,아동·청소년,청년,한부모,1인가구,가족이 특별한 상황에 처했을 때 (다문화가족 지원),저소득층,주거,다문화·북한이탈주민,임신·출산,장애인,중장년', 'title': '긴급복지 지원제도'}),
 Document(page_content='긴급복지 지원제도\n\n대상\n대한민국 국민과 혼인 중인 사람,\n대한민국 국민인 배우자와 이혼 또는 사망으로 대한민국 국적을 가진 부모·조부모, 자녀·손자녀를 돌보고 있는 사람,\n난민으로 인정된 사람,\n본인의 귀책사유 없이 화재, 범죄, 천재지변으로 피해를 입은 사람\n\n내용\n생계지원, 의료지원, 주거지원, 사회복지시설 이용지원, 교육지원, 그 밖의 지원(동절기 연료비, 해산비, 장제비, 전기요금)\n긴급복지 지원제도 참조 (본 

In [50]:
query = "만 45세 이하의 선취업 후진학자가 학부 대출을 받기 위해 충족해야 하는 학점은 몇 학점인가요?"
result = chain(query)
print(result)

{'query': '만 45세 이하의 선취업 후진학자가 학부 대출을 받기 위해 충족해야 하는 학점은 몇 학점인가요?', 'result': '선취업 후진학자는 학부 대출을 받기 위해 직전학기에 12학점 이상을 이수해야 합니다.', 'source_documents': [Document(page_content='일반 상환 학자금 대출\n\n대상\n 대상은 국내 대학 학부생, 대학원생 및 전문기술석사 이수 중인 학생, 학점은행제 교육훈련기관의 평가인정 학습과정 학습자\n 연령은 만 55세 이하\n 성적은 직전학기 성적 70점/100점(C학점) 이상 - 신입생, 장애인은 적용 제외\n 이수학점은 직전학기 12학점 이상 이수 - 신입생, 장애인, 대학원생, 학점은행제 교육훈련기관 학습자는 적용 제외\n 소득은 소득 무관\n 신용은 학자금대출 연체자 및 금융채무 불이행자 등 대출 제한\n\n내용\n 대출금리의 지원내용은 1.7%(고정금리)\n 대출조건의 지원내용은 등록금 대출(당해학기 소요액 전액), 생활비 대출(학기당 150만 원, 단 ’23년도 1학기는 200만 원), 등록금 대출은 학과·학제별 총한도 범위 내 가능하며, 자세한 사항은 한국장학재단에 문의 - 학점은행제 교육훈련기관 학습자는 생활비대출 미지원\n 상환방법dms 원리금균등분할상환 또는 원금균등분할상환(최장 20년 내)', metadata={'doc_id': '070dd740-7acb-4809-8719-50390f71aa73', 'tag': '교육 대출,서민금융,청년,교육비가 부담될 때(학자금대출 지원),교육,학자금', 'title': '일반 상환 학자금 대출'}), Document(page_content='취업 후 상환 학자금대출\n\n대상\n 교육부 또는 한국장학재단과 취업 후 상환 학자금대출 협약을 체결한 국내 고등교육기관 (학점은행제 교육기관, 외국대학 제외)의 학부생, 전문대학의 전문기술석사, 대학원생으로 재학 및 입학·복학 예정인 대한민국 국민(주민등록상 해외이주 신고자, 영주권자 및 재외국민 제

In [51]:
result['source_documents']

[Document(page_content='일반 상환 학자금 대출\n\n대상\n 대상은 국내 대학 학부생, 대학원생 및 전문기술석사 이수 중인 학생, 학점은행제 교육훈련기관의 평가인정 학습과정 학습자\n 연령은 만 55세 이하\n 성적은 직전학기 성적 70점/100점(C학점) 이상 - 신입생, 장애인은 적용 제외\n 이수학점은 직전학기 12학점 이상 이수 - 신입생, 장애인, 대학원생, 학점은행제 교육훈련기관 학습자는 적용 제외\n 소득은 소득 무관\n 신용은 학자금대출 연체자 및 금융채무 불이행자 등 대출 제한\n\n내용\n 대출금리의 지원내용은 1.7%(고정금리)\n 대출조건의 지원내용은 등록금 대출(당해학기 소요액 전액), 생활비 대출(학기당 150만 원, 단 ’23년도 1학기는 200만 원), 등록금 대출은 학과·학제별 총한도 범위 내 가능하며, 자세한 사항은 한국장학재단에 문의 - 학점은행제 교육훈련기관 학습자는 생활비대출 미지원\n 상환방법dms 원리금균등분할상환 또는 원금균등분할상환(최장 20년 내)', metadata={'doc_id': '070dd740-7acb-4809-8719-50390f71aa73', 'tag': '교육 대출,서민금융,청년,교육비가 부담될 때(학자금대출 지원),교육,학자금', 'title': '일반 상환 학자금 대출'}),
 Document(page_content='취업 후 상환 학자금대출\n\n대상\n 교육부 또는 한국장학재단과 취업 후 상환 학자금대출 협약을 체결한 국내 고등교육기관 (학점은행제 교육기관, 외국대학 제외)의 학부생, 전문대학의 전문기술석사, 대학원생으로 재학 및 입학·복학 예정인 대한민국 국민(주민등록상 해외이주 신고자, 영주권자 및 재외국민 제외)\n\n대상의 학부는 협약 체결 대학 학부생, 대학원은 협약 체결 대학 대학원생 및 전문대학의 전문기술석사 이수 중인 학생 |\n 연령의 학부는 만 35세 이하 (선취업 후진학자 등은 만 45세 이하), 대학원은 만 40세 이하\n 성적의 학부는 제한없음, 대

In [52]:
query = "청소년인터넷드림마을의 상설 치유 프로그램은 어떻게 참여하나요?"
result = chain(query)
print(result)

{'query': '청소년인터넷드림마을의 상설 치유 프로그램은 어떻게 참여하나요?', 'result': '청소년인터넷드림마을의 상설 치유 프로그램에 참여하려면 지역별 청소년상담복지센터를 통해 신청해야 합니다. 신청 후에는 인터넷·스마트폰 과의존 정도를 진단평가 받은 후, 과의존 단계별로 지원을 받을 수 있습니다. 자세한 내용은 청소년상담복지센터나 국립청소년인터넷드림마을로 문의하시면 됩니다.\n\n[출처: 청소년 인터넷•스마트폰 과의존 예방 및 해소 지원]', 'source_documents': [Document(page_content='청소년 인터넷•스마트폰 과의존 예방 및 해소 지원\n\n대상\n 만 19세 미만 청소년\n\n내용\n 학령전환기 청소년(초4, 중1, 고1) 대상 인터넷·스마트폰 과의존 상담, 치유 등 지원\n\n상담 및 치유프로그램의 지원내용은 인터넷치유캠프(11박 12일), 가족치유캠프(2박 3일) (기초생활수급자, 차상위, 한부모, 다문화 가족 등은 무료로 참여할 수 있고, 그 외 청소년(가족)은 식비 일부만 부담), 국립청소년인터넷드림마을(상설 인터넷 치유 전담시설)의 상설 치유 프로그램 (1~4주 과정)\n   과의존 고위험군치료비 지원 내용은 일반청소년 : 최대 40만 원, 소득인정액이 기준 중위소득 50%(4인 기준 270만 482원) 이하인 사회적돌봄대상 : 최대 60만 원\n\n방법\n 지역별 청소년상담복지센터를 통해 신청 (인터넷·스마트폰 과의존 정도를 진단평가 후 과의존 단계별 지원)', metadata={'doc_id': 'f6b59afb-9630-42e8-9af3-e711d85c96e1', 'source': '/Users/siina/Desktop/ssis/gitfile/ssis/DB_work_cln/04_청소년•청년_지원/04_청소년_인터넷•스마트폰_과의존_예방_및_해소_지원.md'}), Document(page_content='방법\n 지역별 청소

In [53]:
result['source_documents']

[Document(page_content='청소년 인터넷•스마트폰 과의존 예방 및 해소 지원\n\n대상\n 만 19세 미만 청소년\n\n내용\n 학령전환기 청소년(초4, 중1, 고1) 대상 인터넷·스마트폰 과의존 상담, 치유 등 지원\n\n상담 및 치유프로그램의 지원내용은 인터넷치유캠프(11박 12일), 가족치유캠프(2박 3일) (기초생활수급자, 차상위, 한부모, 다문화 가족 등은 무료로 참여할 수 있고, 그 외 청소년(가족)은 식비 일부만 부담), 국립청소년인터넷드림마을(상설 인터넷 치유 전담시설)의 상설 치유 프로그램 (1~4주 과정)\n   과의존 고위험군치료비 지원 내용은 일반청소년 : 최대 40만 원, 소득인정액이 기준 중위소득 50%(4인 기준 270만 482원) 이하인 사회적돌봄대상 : 최대 60만 원\n\n방법\n 지역별 청소년상담복지센터를 통해 신청 (인터넷·스마트폰 과의존 정도를 진단평가 후 과의존 단계별 지원)', metadata={'doc_id': 'f6b59afb-9630-42e8-9af3-e711d85c96e1', 'source': '/Users/siina/Desktop/ssis/gitfile/ssis/DB_work_cln/04_청소년•청년_지원/04_청소년_인터넷•스마트폰_과의존_예방_및_해소_지원.md'}),
 Document(page_content='방법\n 지역별 청소년상담복지센터를 통해 신청 (인터넷·스마트폰 과의존 정도를 진단평가 후 과의존 단계별 지원)\n\n문의\n 전국 청소년상담복지센터(청소년전화 1388, 휴대전화는 지역번호+1388)\n 한국청소년상담복지개발원(전화 051-662-3191~5)\n 국립청소년인터넷드림마을(전화 063-323-2285)', metadata={'doc_id': 'f6b59afb-9630-42e8-9af3-e711d85c96e1', 'source': '/Users/siina/Desktop/ssis/gitfile/ssi

## test

In [23]:
query = "농촌출신 학자금 지원 사업에 대해 알려줘."
result = chain(query)
print(result['source_documents'])

[Document(page_content='농촌출신 대학생 학자금 융자사업\n\n대상\n 농어촌지역에 주민등록 주소를 두고 6개월(180일) 이상 거주하고 있거나 농어업에 종사하고 있는 학부모(보호자)의 자녀(대학생) 또는 농어업에 종사하는 대학생 본인\n 직전학기 성적 70점/100점(C학점) 이상, 12학점 이상 이수한 대학생\n   소속대학의 최저 이수학점이 12학점 미만인 경우는 대학 학사규정에 의함\n   신입생군(신입생, 편입생, 재입학생), 장애인인 경우 성적, 이수학점 기준 적용 제외\n   졸업 학년 학생의 경우 이수학점 기준 적용 제외\n\n1순위의 세부 자격 요건은 농어업에 종사하는 학부모(보호자)의 자녀(대학생), 농어업에 종사하는 대학생 본인, 기초생활수급자, 차상위계층, 장애인, 다문화가족, 다자녀가구(3자녀 이상)\n 2순위의 세부 자격 요건은 농어촌지역에 거주하는 학부모(보호자)의 자녀(대학생)\n 3순위의 세부 자격 요건은 특별승인자\n\n내용\n 당해 학기 대학이 통보한 등록금(입학금, 수업료 등) 범위 내 전액 무이자 대출', metadata={'doc_id': '151f20a4-7a77-4f5f-9c94-c573e80c92a6', 'tag': '교육 대출,서민금융,청년,교육비가 부담될 때(학자금대출 지원),교육,기타위기별·상황별 지원,농어업인·도서민,학자금', 'title': '농촌출신 대학생 학자금 융자사업'}), Document(page_content='내용\n 당해 학기 대학이 통보한 등록금(입학금, 수업료 등) 범위 내 전액 무이자 대출\n\n방법\n 한국장학재단 누리집(https://www.kosaf.go.kr)에서 신청\n\n문의\n 한국장학재단(전화 1599-2290, www.kosaf.go.kr)\n\n자주 하는 질문\n 농촌학자금융자를 신청하려고 합니다. 나이제한이 있나요?\n   농촌학자금융자는 나이제한은 없으며, 대한민국 국적인 대학(교) 학부생이면 됩니다\n 농촌학자금융자는 생활비 대출은 없나요?\n  

In [27]:
vectorstore.similarity_search_with_score(query)

[(Document(page_content='농촌출신 대학생 학자금 융자사업\n\n대상\n 농어촌지역에 주민등록 주소를 두고 6개월(180일) 이상 거주하고 있거나 농어업에 종사하고 있는 학부모(보호자)의 자녀(대학생) 또는 농어업에 종사하는 대학생 본인\n 직전학기 성적 70점/100점(C학점) 이상, 12학점 이상 이수한 대학생\n   소속대학의 최저 이수학점이 12학점 미만인 경우는 대학 학사규정에 의함\n   신입생군(신입생, 편입생, 재입학생), 장애인인 경우 성적, 이수학점 기준 적용 제외\n   졸업 학년 학생의 경우 이수학점 기준 적용 제외\n\n1순위의 세부 자격 요건은 농어업에 종사하는 학부모(보호자)의 자녀(대학생), 농어업에 종사하는 대학생 본인, 기초생활수급자, 차상위계층, 장애인, 다문화가족, 다자녀가구(3자녀 이상)\n 2순위의 세부 자격 요건은 농어촌지역에 거주하는 학부모(보호자)의 자녀(대학생)\n 3순위의 세부 자격 요건은 특별승인자\n\n내용\n 당해 학기 대학이 통보한 등록금(입학금, 수업료 등) 범위 내 전액 무이자 대출', metadata={'doc_id': '151f20a4-7a77-4f5f-9c94-c573e80c92a6', 'tag': '교육 대출,서민금융,청년,교육비가 부담될 때(학자금대출 지원),교육,기타위기별·상황별 지원,농어업인·도서민,학자금', 'title': '농촌출신 대학생 학자금 융자사업'}),
  0.5733091235160828),
 (Document(page_content='내용\n 당해 학기 대학이 통보한 등록금(입학금, 수업료 등) 범위 내 전액 무이자 대출\n\n방법\n 한국장학재단 누리집(https://www.kosaf.go.kr)에서 신청\n\n문의\n 한국장학재단(전화 1599-2290, www.kosaf.go.kr)\n\n자주 하는 질문\n 농촌학자금융자를 신청하려고 합니다. 나이제한이 있나요?\n   농촌학자금융자는 나이제한은 없으며, 대한민국 국적인 대학(교) 학부생이면 됩니다\n

In [42]:
query = "청소년인터넷드림마을의 상설 치유 프로그램은 어떻게 참여하나요?"
result = chain(query)
print(result['source_documents'])

[Document(page_content='청소년 인터넷•스마트폰 과의존 예방 및 해소 지원\n\n대상\n 만 19세 미만 청소년\n\n내용\n 학령전환기 청소년(초4, 중1, 고1) 대상 인터넷·스마트폰 과의존 상담, 치유 등 지원\n\n상담 및 치유프로그램의 지원내용은 인터넷치유캠프(11박 12일), 가족치유캠프(2박 3일) (기초생활수급자, 차상위, 한부모, 다문화 가족 등은 무료로 참여할 수 있고, 그 외 청소년(가족)은 식비 일부만 부담), 국립청소년인터넷드림마을(상설 인터넷 치유 전담시설)의 상설 치유 프로그램 (1~4주 과정)\n   과의존 고위험군치료비 지원 내용은 일반청소년 : 최대 40만 원, 소득인정액이 기준 중위소득 50%(4인 기준 270만 482원) 이하인 사회적돌봄대상 : 최대 60만 원\n\n방법\n 지역별 청소년상담복지센터를 통해 신청 (인터넷·스마트폰 과의존 정도를 진단평가 후 과의존 단계별 지원)', metadata={'doc_id': 'f6b59afb-9630-42e8-9af3-e711d85c96e1', 'source': '/Users/siina/Desktop/ssis/gitfile/ssis/DB_work_cln/04_청소년•청년_지원/04_청소년_인터넷•스마트폰_과의존_예방_및_해소_지원.md'}), Document(page_content='방법\n 지역별 청소년상담복지센터를 통해 신청 (인터넷·스마트폰 과의존 정도를 진단평가 후 과의존 단계별 지원)\n\n문의\n 전국 청소년상담복지센터(청소년전화 1388, 휴대전화는 지역번호+1388)\n 한국청소년상담복지개발원(전화 051-662-3191~5)\n 국립청소년인터넷드림마을(전화 063-323-2285)', metadata={'doc_id': 'f6b59afb-9630-42e8-9af3-e711d85c96e1', 'source': '/Users/siina/Desktop/ssis/gitfile/ssis

In [47]:
result['source_documents'][0].metadata['source']

'/Users/siina/Desktop/ssis/gitfile/ssis/DB_work_cln/04_청소년•청년_지원/04_청소년_인터넷•스마트폰_과의존_예방_및_해소_지원.md'

In [33]:
vectorstore.similarity_search_with_score(query)

[(Document(page_content='청소년 인터넷•스마트폰 과의존 예방 및 해소 지원\n\n대상\n 만 19세 미만 청소년\n\n내용\n 학령전환기 청소년(초4, 중1, 고1) 대상 인터넷·스마트폰 과의존 상담, 치유 등 지원\n\n상담 및 치유프로그램의 지원내용은 인터넷치유캠프(11박 12일), 가족치유캠프(2박 3일) (기초생활수급자, 차상위, 한부모, 다문화 가족 등은 무료로 참여할 수 있고, 그 외 청소년(가족)은 식비 일부만 부담), 국립청소년인터넷드림마을(상설 인터넷 치유 전담시설)의 상설 치유 프로그램 (1~4주 과정)\n   과의존 고위험군치료비 지원 내용은 일반청소년 : 최대 40만 원, 소득인정액이 기준 중위소득 50%(4인 기준 270만 482원) 이하인 사회적돌봄대상 : 최대 60만 원\n\n방법\n 지역별 청소년상담복지센터를 통해 신청 (인터넷·스마트폰 과의존 정도를 진단평가 후 과의존 단계별 지원)', metadata={'doc_id': 'f6b59afb-9630-42e8-9af3-e711d85c96e1', 'source': '/Users/siina/Desktop/ssis/gitfile/ssis/DB_work_cln/04_청소년•청년_지원/04_청소년_인터넷•스마트폰_과의존_예방_및_해소_지원.md'}),
  0.5713016390800476),
 (Document(page_content='방법\n 지역별 청소년상담복지센터를 통해 신청 (인터넷·스마트폰 과의존 정도를 진단평가 후 과의존 단계별 지원)\n\n문의\n 전국 청소년상담복지센터(청소년전화 1388, 휴대전화는 지역번호+1388)\n 한국청소년상담복지개발원(전화 051-662-3191~5)\n 국립청소년인터넷드림마을(전화 063-323-2285)', metadata={'doc_id': 'f6b59afb-9630-42e8-9af3-e711d85c96e1', 'source': '/Users/siina

In [48]:
query = "도움을 받고자 하는 청소년이 주거지 인근에 없는 경우 어떻게 상담을 받을 수 있나요?"
result = chain(query)
print(result['source_documents'])

[Document(page_content='청소년동반자 프로그램\n\n대상\n다양한 위기를 겪고 있는 만 9~24세 청소년 (우선지원대상 : 우울증, 자살위험, 가출, 비행, 인터넷 중독, 폭력(성·학교·가정) 피해, 학업중단 청소년 등)\n\n내용\n 청소년 상담 전문가가 위기상황 청소년을 직접 찾아가 상담하고, 청소년의 문제 유형과 개별 특성을 고려한 서비스 제공\n\n방법\n 청소년상담전화 또는 청소년상담복지센터 방문 상담\n\n문의\n 청소년전화(전화 1388, 휴대전화는 국번+1388)\n 문자(1388), 카카오톡·페이스북(청소년상담1388), 채팅 및 게시판 상담(www.cyber1388.kr)', metadata={'doc_id': '497c6522-feab-4405-8a86-a93f537238fc', 'tag': '아동·청소년,보호,보호나 지원이 필요할 때(청소년 보호 및 상담)', 'title': '청소년동반자 프로그램'}), Document(page_content='지역사회 청소년 통합지원체계(청소년안전망)\n\n대상\n 만 9세~24세 위기청소년 및 가족\n\n내용\n 위기청소년이 가정, 학교 및 사회로 안전하게 돌아갈 수 있도록 상담·보호·교육·자립 등 맞춤형 서비스 제공(청소년상담복지센터와 지역사회 청소년 자원 연계·협력)\n (지역사회청소년 통합지원체계(청소년안전망) : 위기청소년 통합지원 및 연계협력, 상담 및 활동지원, 청소년전화 1388 운영, 긴급구조, 일시보호, 교육 및 자립 등 맞춤형 서비스 제공)\n\n방법\n 청소년상담복지센터 전화 또는 방문\n\n문의\n 청소년전화(전화 1388, 휴대전화는 국번+1388)\n 문자(1388), 카카오톡·페이스북(청소년상담1388)\n 채팅 및 게시판 상담(www.cyber1388.kr)', metadata={'doc_id': '9e7ceff2-a3b8-49d8-8313-270adf60cdac', 'tag': '아동·청소년,보호,보호나 지원이 필요할 때(청소년 보호 및 상담)', 't

In [41]:
result['source_documents'][0].metadata['title']

'청소년동반자 프로그램'

In [35]:
vectorstore.similarity_search_with_score(query)

[(Document(page_content='청소년동반자 프로그램\n\n대상\n다양한 위기를 겪고 있는 만 9~24세 청소년 (우선지원대상 : 우울증, 자살위험, 가출, 비행, 인터넷 중독, 폭력(성·학교·가정) 피해, 학업중단 청소년 등)\n\n내용\n 청소년 상담 전문가가 위기상황 청소년을 직접 찾아가 상담하고, 청소년의 문제 유형과 개별 특성을 고려한 서비스 제공\n\n방법\n 청소년상담전화 또는 청소년상담복지센터 방문 상담\n\n문의\n 청소년전화(전화 1388, 휴대전화는 국번+1388)\n 문자(1388), 카카오톡·페이스북(청소년상담1388), 채팅 및 게시판 상담(www.cyber1388.kr)', metadata={'doc_id': '497c6522-feab-4405-8a86-a93f537238fc', 'tag': '아동·청소년,보호,보호나 지원이 필요할 때(청소년 보호 및 상담)', 'title': '청소년동반자 프로그램'}),
  0.5392389297485352),
 (Document(page_content='지역사회 청소년 통합지원체계(청소년안전망)\n\n대상\n 만 9세~24세 위기청소년 및 가족\n\n내용\n 위기청소년이 가정, 학교 및 사회로 안전하게 돌아갈 수 있도록 상담·보호·교육·자립 등 맞춤형 서비스 제공(청소년상담복지센터와 지역사회 청소년 자원 연계·협력)\n (지역사회청소년 통합지원체계(청소년안전망) : 위기청소년 통합지원 및 연계협력, 상담 및 활동지원, 청소년전화 1388 운영, 긴급구조, 일시보호, 교육 및 자립 등 맞춤형 서비스 제공)\n\n방법\n 청소년상담복지센터 전화 또는 방문\n\n문의\n 청소년전화(전화 1388, 휴대전화는 국번+1388)\n 문자(1388), 카카오톡·페이스북(청소년상담1388)\n 채팅 및 게시판 상담(www.cyber1388.kr)', metadata={'doc_id': '9e7ceff2-a3b8-49d8-8313-270adf60cdac', 'tag': '아동·청소년,보호,보호나 지

In [36]:
query = "이동에 어려움을 가진 장애인을 위해 어떤 서비스를 지원하나요?"
result = chain(query)
print(result['source_documents'])

[Document(page_content='장애인 생활이동지원센터\n\n대상\n등록장애인\n\n내용\n이동에 제약이 있는 장애인의 민원업무 보조, 직장 출퇴근, 시장보기, 외출보조 및 병원 이용보조 등 각종 이동서비스 및 안내를 보조하기 위해 차량운행 지원\n\n방법\n해당 지역 장애인 생활이동지원센터에 신청\n\n문의\n각 시도(시군구) 장애인생활이동지원센터', metadata={'doc_id': '179600ca-eaa2-4105-b9e6-8096072be484', 'tag': '지역사회 복지시설을 이용하고 싶을 때,장애인,돌봄 (안전·활동지원)', 'title': '장애인 생활이동지원센터'}), Document(page_content='5. 장애인·노인을 위한 돌봄여행: 전문 돌봄 인력이 동반하는 여행 서비스를 제공합니다. 이 서비스는 등록장애인(또는 상이 등급자)과 만 65세 이상의 노인을 대상으로 하며, 기준 중위소득 140% 이하인 가구를 지원합니다.\n6. 장애인 보조기기 렌탈서비스: 만 24세 이하의 장애 아동과 청소년의 성장 단계에 맞는 맞춤형 보조기기 렌탈 서비스를 제공합니다. 이 서비스는 만 24세 이하의 장애 아동과 청소년을 대상으로 하며, 소득기준은 없습니다.', metadata={'doc_id': '77911b96-69b7-4c20-aecd-128d21c0ff0c', 'tag': '기타,더욱 다양한 서비스를 원할 때,기타위기별·상황별 지원', 'title': '지역사회서비스 투자사업'}), Document(page_content='지체·뇌병변·심장·호흡 장애인은 미끄럼 및 회전을 위한 보조기기, 장애인용 의복 및 신발, 휠체어 액세서리, 전동침대, 소변수집장치, 낙상알림기, 독서용 탁자, 책상 및 독서대 등을 지원받을 수 있습니다.\n뇌병변·지적·자폐성·청각·언어 장애인은 대화용장치를 지원받을 수 있습니다.\n뇌병변 장애인은 대체입력장치 (스위치)를 지원받을 수 있습니다.\n지체·뇌병변·지적·자폐 장애인은 기억 지원 보조기기를 지원받을

In [37]:
vectorstore.similarity_search_with_score(query)

[(Document(page_content='장애인 생활이동지원센터\n\n대상\n등록장애인\n\n내용\n이동에 제약이 있는 장애인의 민원업무 보조, 직장 출퇴근, 시장보기, 외출보조 및 병원 이용보조 등 각종 이동서비스 및 안내를 보조하기 위해 차량운행 지원\n\n방법\n해당 지역 장애인 생활이동지원센터에 신청\n\n문의\n각 시도(시군구) 장애인생활이동지원센터', metadata={'doc_id': '179600ca-eaa2-4105-b9e6-8096072be484', 'tag': '지역사회 복지시설을 이용하고 싶을 때,장애인,돌봄 (안전·활동지원)', 'title': '장애인 생활이동지원센터'}),
  0.4301528334617615),
 (Document(page_content='5. 장애인·노인을 위한 돌봄여행: 전문 돌봄 인력이 동반하는 여행 서비스를 제공합니다. 이 서비스는 등록장애인(또는 상이 등급자)과 만 65세 이상의 노인을 대상으로 하며, 기준 중위소득 140% 이하인 가구를 지원합니다.\n6. 장애인 보조기기 렌탈서비스: 만 24세 이하의 장애 아동과 청소년의 성장 단계에 맞는 맞춤형 보조기기 렌탈 서비스를 제공합니다. 이 서비스는 만 24세 이하의 장애 아동과 청소년을 대상으로 하며, 소득기준은 없습니다.', metadata={'doc_id': '77911b96-69b7-4c20-aecd-128d21c0ff0c', 'tag': '기타,더욱 다양한 서비스를 원할 때,기타위기별·상황별 지원', 'title': '지역사회서비스 투자사업'}),
  0.8200606107711792),
 (Document(page_content='지체·뇌병변·심장·호흡 장애인은 미끄럼 및 회전을 위한 보조기기, 장애인용 의복 및 신발, 휠체어 액세서리, 전동침대, 소변수집장치, 낙상알림기, 독서용 탁자, 책상 및 독서대 등을 지원받을 수 있습니다.\n뇌병변·지적·자폐성·청각·언어 장애인은 대화용장치를 지원받을 수 있습니다.\n뇌병변 장애인은 대체입력장치 (스위치

In [38]:
query = "만 45세 이하의 선취업 후진학자가 학부 대출을 받기 위해 충족해야 하는 학점은 몇 학점인가요?"
result = chain(query)
print(result['source_documents'])

[Document(page_content='일반 상환 학자금 대출\n\n대상\n 대상은 국내 대학 학부생, 대학원생 및 전문기술석사 이수 중인 학생, 학점은행제 교육훈련기관의 평가인정 학습과정 학습자\n 연령은 만 55세 이하\n 성적은 직전학기 성적 70점/100점(C학점) 이상 - 신입생, 장애인은 적용 제외\n 이수학점은 직전학기 12학점 이상 이수 - 신입생, 장애인, 대학원생, 학점은행제 교육훈련기관 학습자는 적용 제외\n 소득은 소득 무관\n 신용은 학자금대출 연체자 및 금융채무 불이행자 등 대출 제한\n\n내용\n 대출금리의 지원내용은 1.7%(고정금리)\n 대출조건의 지원내용은 등록금 대출(당해학기 소요액 전액), 생활비 대출(학기당 150만 원, 단 ’23년도 1학기는 200만 원), 등록금 대출은 학과·학제별 총한도 범위 내 가능하며, 자세한 사항은 한국장학재단에 문의 - 학점은행제 교육훈련기관 학습자는 생활비대출 미지원\n 상환방법dms 원리금균등분할상환 또는 원금균등분할상환(최장 20년 내)', metadata={'doc_id': '070dd740-7acb-4809-8719-50390f71aa73', 'tag': '교육 대출,서민금융,청년,교육비가 부담될 때(학자금대출 지원),교육,학자금', 'title': '일반 상환 학자금 대출'}), Document(page_content='취업 후 상환 학자금대출\n\n대상\n 교육부 또는 한국장학재단과 취업 후 상환 학자금대출 협약을 체결한 국내 고등교육기관 (학점은행제 교육기관, 외국대학 제외)의 학부생, 전문대학의 전문기술석사, 대학원생으로 재학 및 입학·복학 예정인 대한민국 국민(주민등록상 해외이주 신고자, 영주권자 및 재외국민 제외)\n\n대상의 학부는 협약 체결 대학 학부생, 대학원은 협약 체결 대학 대학원생 및 전문대학의 전문기술석사 이수 중인 학생 |\n 연령의 학부는 만 35세 이하 (선취업 후진학자 등은 만 45세 이하), 대학원은 만 40세 이하\n 성적의 학부는 제한없음, 대학

In [39]:
vectorstore.similarity_search_with_score(query)

[(Document(page_content='일반 상환 학자금 대출\n\n대상\n 대상은 국내 대학 학부생, 대학원생 및 전문기술석사 이수 중인 학생, 학점은행제 교육훈련기관의 평가인정 학습과정 학습자\n 연령은 만 55세 이하\n 성적은 직전학기 성적 70점/100점(C학점) 이상 - 신입생, 장애인은 적용 제외\n 이수학점은 직전학기 12학점 이상 이수 - 신입생, 장애인, 대학원생, 학점은행제 교육훈련기관 학습자는 적용 제외\n 소득은 소득 무관\n 신용은 학자금대출 연체자 및 금융채무 불이행자 등 대출 제한\n\n내용\n 대출금리의 지원내용은 1.7%(고정금리)\n 대출조건의 지원내용은 등록금 대출(당해학기 소요액 전액), 생활비 대출(학기당 150만 원, 단 ’23년도 1학기는 200만 원), 등록금 대출은 학과·학제별 총한도 범위 내 가능하며, 자세한 사항은 한국장학재단에 문의 - 학점은행제 교육훈련기관 학습자는 생활비대출 미지원\n 상환방법dms 원리금균등분할상환 또는 원금균등분할상환(최장 20년 내)', metadata={'doc_id': '070dd740-7acb-4809-8719-50390f71aa73', 'tag': '교육 대출,서민금융,청년,교육비가 부담될 때(학자금대출 지원),교육,학자금', 'title': '일반 상환 학자금 대출'}),
  0.5217203497886658),
 (Document(page_content='취업 후 상환 학자금대출\n\n대상\n 교육부 또는 한국장학재단과 취업 후 상환 학자금대출 협약을 체결한 국내 고등교육기관 (학점은행제 교육기관, 외국대학 제외)의 학부생, 전문대학의 전문기술석사, 대학원생으로 재학 및 입학·복학 예정인 대한민국 국민(주민등록상 해외이주 신고자, 영주권자 및 재외국민 제외)\n\n대상의 학부는 협약 체결 대학 학부생, 대학원은 협약 체결 대학 대학원생 및 전문대학의 전문기술석사 이수 중인 학생 |\n 연령의 학부는 만 35세 이하 (선취업 후진학자 등은 만 45세 이하), 대학원은 만

In [13]:
query = "우리집은 가난해. 우리 집에 화재가 나서 화상을 입었어. 위로금 같은걸 받을 수 있을까?"
vectorstore.similarity_search_with_score(query)

[(Document(page_content='긴급복지 지원제도\n\n대상\n대한민국 국민과 혼인 중인 사람,\n대한민국 국민인 배우자와 이혼 또는 사망으로 대한민국 국적을 가진 부모·조부모, 자녀·손자녀를 돌보고 있는 사람,\n난민으로 인정된 사람,\n본인의 귀책사유 없이 화재, 범죄, 천재지변으로 피해를 입은 사람\n\n내용\n생계지원, 의료지원, 주거지원, 사회복지시설 이용지원, 교육지원, 그 밖의 지원(동절기 연료비, 해산비, 장제비, 전기요금)\n긴급복지 지원제도 참조 (본 책자 20p)\n\n방법\n시·군·구청, 읍·면·동 주민센터(행정복지센터) 및 보건복지상담센터(전화 129)에 지원요청 또는 신고\n\n문의\n보건복지상담센터(전화 129)', metadata={'doc_id': '5a3d0bd4-30f0-4066-b547-1196d93e3ef4', 'tag': '영유아,노령층,건강,생활지원,아동·청소년,청년,한부모,1인가구,가족이 특별한 상황에 처했을 때 (다문화가족 지원),저소득층,주거,다문화·북한이탈주민,임신·출산,장애인,중장년', 'title': '긴급복지 지원제도'}),
  0.9627849459648132),
 (Document(page_content="주소득자가 사망, 가출, 행방불명, 구금시설에 수용되는 등의 사유로 소득을 상실한 경우, 주소득자 또는 부소득자의 실직, 휴업, 폐업 또는 사업장의 화재, 중한 질병이나 부상 등으로 소득 상실 또는 소득활동이 중단된 경우, 가구구성원으로부터 방임 또는 유기되거나 학대 등을 당한 경우, 가정폭력을 당하여 가구구성원과 함께 원만한 가정생활을 하기 곤란하거나 가구구성원으로부터 성폭력을 당한 경우, 화재 또는 자연재해 등으로 인하여 거주하는 주택 또는 건물에서 생활하기 곤란하게 된 경우, 주소득자와 이혼 또는 이혼 소송 등 사실상 가족관계 단절로 생계가 곤란한 경우, 자살한 자의 유족, 자살을 시도한 자 또는 그의 가족인 자살 고위험군으로서 생계가 곤란한 경우 (단, 심리지원서비스대상자로 동의한 

In [14]:
query = "도움을 받고자 하는 청소년이 주거지 인근에 없는 경우 어떻게 상담을 받을 수 있나요?"
vectorstore.similarity_search_with_score(query)

[(Document(page_content='지역사회 청소년 통합지원체계(청소년안전망)\n\n대상\n 만 9세~24세 위기청소년 및 가족\n\n내용\n 위기청소년이 가정, 학교 및 사회로 안전하게 돌아갈 수 있도록 상담·보호·교육·자립 등 맞춤형 서비스 제공(청소년상담복지센터와 지역사회 청소년 자원 연계·협력)\n (지역사회청소년 통합지원체계(청소년안전망) : 위기청소년 통합지원 및 연계협력, 상담 및 활동지원, 청소년전화 1388 운영, 긴급구조, 일시보호, 교육 및 자립 등 맞춤형 서비스 제공)\n\n방법\n 청소년상담복지센터 전화 또는 방문\n\n문의\n 청소년전화(전화 1388, 휴대전화는 국번+1388)\n 문자(1388), 카카오톡·페이스북(청소년상담1388)\n 채팅 및 게시판 상담(www.cyber1388.kr)', metadata={'doc_id': '9e7ceff2-a3b8-49d8-8313-270adf60cdac', 'tag': '아동·청소년,보호,보호나 지원이 필요할 때(청소년 보호 및 상담)', 'title': '지역사회 청소년 통합지원체계(청소년안전망)'}),
  1.0615203380584717),
 (Document(page_content='청소년쉼터\n\n대상\n 만 9세~24세 위기청소년 및 가족\n\n내용\n 위기청소년이 가정, 학교 및 사회로 안전하게 돌아갈 수 있도록 상담·보호·교육·자립 등 맞춤형 서비스 제공(청소년상담복지센터와 지역사회 청소년 자원 연계·협력)\n  입소 중 내용은 의식주 등 생활보호, 의료 및 법률자문 지원 연계, 진로탐색 및 직업지도, 직업훈련 및 취업지원, 검정고시, 학업복귀 지원 등 개별 가정 밖 청소년이 필요로 하는 맞춤형 서비스 지원\n  퇴소 후 내용은 LH임대주택 우선지원대상으로 지원 (쉼터를 2년 이상 이용하고, 퇴소 후 5년 이내인 18세 이상 미혼 무주택자(건설임대는 나이 및 결혼 여부 관계 없음)), 자립지원수당 지원 (만 18세 이후 퇴소자(’21.1월 이후 퇴소)로서 퇴소일로부터